In [21]:
import pandas as pd
import numpy as np
import panel as pn
import hvplot.pandas
import holoviews as hv
pn.extension('tabulator', 'plotly')

import plotly.graph_objects as go
from plotly.subplots import make_subplots

from typing import List, Dict
import logging

import os
from pathlib import Path

### Data Processing for Career Timeline




In [9]:
def combine_csv_files(folder_path):
    """
    Read all CSV files from a folder and combine them into a single DataFrame.
    
    Parameters:
    folder_path (str): Path to the folder containing CSV files
    
    Returns:
    pandas.DataFrame: Combined DataFrame from all CSV files
    """
    path = Path(folder_path)
    all_dfs = []
    
    # Loop through all files in the folder
    for file in path.glob('*.csv'):
        try:
            df = pd.read_csv(file)
            # Append to the list
            all_dfs.append(df)
            
            print(f"Successfully read: {file.name}")
            
        except Exception as e:
            print(f"Error reading {file.name}: {str(e)}")
    
    # Combine all DataFrames
    if all_dfs:
        combined_df = pd.concat(all_dfs, ignore_index=True)
        print(f"\nTotal number of files combined: {len(all_dfs)}")
        print(f"Total rows in combined DataFrame: {len(combined_df)}")
        return combined_df
    else:
        print("No CSV files found in the specified folder!")
        return None

In [10]:
quali_data = combine_csv_files('data/results_data')
print(quali_data['Year'].unique())
quali_data.head()

Successfully read: qualifying_data_2019_results.csv
Successfully read: qualifying_data_2022_results.csv
Successfully read: qualifying_data_2021_results.csv
Successfully read: qualifying_data_2018_results.csv
Successfully read: qualifying_data_2020_results.csv

Total number of files combined: 5
Total rows in combined DataFrame: 1900
[2019 2022 2021 2018 2020]


,DriverNumber,BroadcastName,TeamName,Position,Q1,Q2,Q3,Year,EventName,WetSession
0,44,L HAMILTON,Mercedes,1.0,0 days 00:01:22.043000,0 days 00:01:21.014000,0 days 00:01:20.486000,2019,Australian Grand Prix,False
1,77,V BOTTAS,Mercedes,2.0,0 days 00:01:22.367000,0 days 00:01:21.193000,0 days 00:01:20.598000,2019,Australian Grand Prix,False
2,5,S VETTEL,Ferrari,3.0,0 days 00:01:22.885000,0 days 00:01:21.912000,0 days 00:01:21.190000,2019,Australian Grand Prix,False
3,33,M VERSTAPPEN,Red Bull Racing,4.0,0 days 00:01:22.876000,0 days 00:01:21.678000,0 days 00:01:21.320000,2019,Australian Grand Prix,False
4,16,C LECLERC,Ferrari,5.0,0 days 00:01:22.017000,0 days 00:01:21.739000,0 days 00:01:21.442000,2019,Australian Grand Prix,False


In [11]:
print(quali_data[['Q1', 'Q2', 'Q3']].dtypes)

Q1    object
Q2    object
Q3    object
dtype: object


In [12]:
# Convert from string to Timedelta
quali_data['Q1'] = pd.to_timedelta(quali_data['Q1'])
quali_data['Q2'] = pd.to_timedelta(quali_data['Q2'])
quali_data['Q3'] = pd.to_timedelta(quali_data['Q3'])

# Now convert the Timedelta columns to total seconds
quali_data['Q1Seconds'] = quali_data['Q1'].apply(lambda x: x.total_seconds())
quali_data['Q2Seconds'] = quali_data['Q2'].apply(lambda x: x.total_seconds())
quali_data['Q3Seconds'] = quali_data['Q3'].apply(lambda x: x.total_seconds())

quali_data.head()

,DriverNumber,BroadcastName,TeamName,Position,Q1,Q2,Q3,Year,EventName,WetSession,Q1Seconds,Q2Seconds,Q3Seconds
0,44,L HAMILTON,Mercedes,1.0,0 days 00:01:22.043000,0 days 00:01:21.014000,0 days 00:01:20.486000,2019,Australian Grand Prix,False,82.043,81.014,80.486
1,77,V BOTTAS,Mercedes,2.0,0 days 00:01:22.367000,0 days 00:01:21.193000,0 days 00:01:20.598000,2019,Australian Grand Prix,False,82.367,81.193,80.598
2,5,S VETTEL,Ferrari,3.0,0 days 00:01:22.885000,0 days 00:01:21.912000,0 days 00:01:21.190000,2019,Australian Grand Prix,False,82.885,81.912,81.190
3,33,M VERSTAPPEN,Red Bull Racing,4.0,0 days 00:01:22.876000,0 days 00:01:21.678000,0 days 00:01:21.320000,2019,Australian Grand Prix,False,82.876,81.678,81.320
4,16,C LECLERC,Ferrari,5.0,0 days 00:01:22.017000,0 days 00:01:21.739000,0 days 00:01:21.442000,2019,Australian Grand Prix,False,82.017,81.739,81.442


In [13]:
quali_data.columns

Index(['DriverNumber', 'BroadcastName', 'TeamName', 'Position', 'Q1', 'Q2',
       'Q3', 'Year', 'EventName', 'WetSession', 'Q1Seconds', 'Q2Seconds',
       'Q3Seconds'],
      dtype='object')

### Calculate Metrics for Career Timeline

In [47]:

from typing import List, Dict
import logging

# Set up logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)


def process_qualifying_data(quali_data: pd.DataFrame) -> List[Dict]:
    """
    Process F1 qualifying data .
    
    """
    timeline_data = []
    
    # Verify required columns
    required_columns = ['DriverNumber', 'BroadcastName', 'TeamName', 'Position', 'Q1', 'Q2',
       'Q3', 'Year', 'EventName', 'WetSession', 'Q1Seconds', 'Q2Seconds',
       'Q3Seconds']
    missing_columns = [col for col in required_columns if col not in quali_data.columns]
    if missing_columns:
        raise ValueError(f"Missing required columns: {missing_columns}")
    
    for year in quali_data['Year'].unique():
        year_data = quali_data[quali_data['Year'] == year]
        
        for event_name, event_data in year_data.groupby('EventName'):
            #logger.info(f"Processing {event_name} {year}")
            
            # Get pole position data (P1)
            pole_data = event_data[event_data['Position'] == 1].iloc[0]
            #pole_driver = pole_data['BroadcastName']
            pole_time = pole_data['Q3Seconds']
            
            team_groups = event_data.groupby('TeamName')
            
            for team, team_data in team_groups:
                drivers = team_data['BroadcastName'].unique()
                
                # Initialize gaps dict
                gaps = {driver: np.nan for driver in drivers}
                
                # Calculate teammate gaps only if we have exactly 2 drivers
                if len(drivers) == 2:
                    driver1, driver2 = drivers
                    time1_data = team_data[team_data['BroadcastName'] == driver1].iloc[0]
                    time2_data = team_data[team_data['BroadcastName'] == driver2].iloc[0]
                    
                    time1 = None
                    if pd.notna(time1_data['Q3Seconds']):
                        time1 = time1_data['Q3Seconds']
                    elif pd.notna(time1_data['Q2Seconds']):
                        time1 = time1_data['Q2Seconds']
                    elif pd.notna(time1_data['Q1Seconds']):
                        time1 = time1_data['Q1Seconds']

                        
                    time2 = None  
                    if pd.notna(time2_data['Q3Seconds']):
                        time2 = time2_data['Q3Seconds']
                    elif pd.notna(time2_data['Q2Seconds']):
                        time2 = time2_data['Q2Seconds']
                    elif pd.notna(time2_data['Q1Seconds']):
                        time2 = time2_data['Q1Seconds']

                    if time1 is not None and time2 is not None:
                        gaps.update({                  #check meaning
                            driver1: time1 - time2,
                            driver2: time2 - time1
                        })
                    else:
                        gaps.update({                  #check meaning
                            driver1: 'N/A',
                            driver2: 'N/A'
                        })
                else:
                    #pass
                    logger.info(f'Incomplete teammate data: {len(drivers)} driver(s) for {team} at {event_name} {year}')
                
                # Process each driver
                for driver in drivers:
                    driver_data = team_data[team_data['BroadcastName'] == driver].iloc[0]
                    best_time = None

                    if pd.notna(driver_data['Q3Seconds']):
                        best_time = driver_data['Q3Seconds']
                    elif pd.notna(driver_data['Q2Seconds']):
                        best_time = driver_data['Q2Seconds']
                    elif pd.notna(driver_data['Q1Seconds']):
                        best_time = driver_data['Q1Seconds']

                    qualifying_position_display = 'N/A' if best_time is None else driver_data['Position']
                    qualifying_position_value = np.nan if best_time is None else driver_data['Position']

                    
                    # Calculate gap to pole
                    if qualifying_position_value == 1:
                        gap_to_pole = 0.0
                    elif best_time is None:
                        gap_to_pole = 'N/A'
                    else:
                        gap_to_pole = best_time - pole_time 
                
                        
                        #if gap_to_pole < 0:
                          #  logger.error(
                             ##   f"Negative gap detected at {event_name} {year}:\n"
                               # f"Driver: {driver} (P{qualifying_position})\n"
                                #f"Driver: {driver} (P{qualifying_position})\n"
                            #    f"Best lap: {driver_data['Q3'] if pd.notna(driver_data['Q3']) else driver_data['Q2'] if pd.notna(driver_data['Q2']) else driver_data['Q1']}\n"
                             #   f"Pole time: {pole_data['Q3']}\n"
                             #   f"Gap: {gap_to_pole:.3f}s\n")

                          #  gap_to_pole = np.nan  # Set invalid gaps to NaN

                    #if best_time == None:
                        #best_time = 'No Lap Set'
                    
                    # Create event summary
                    event_summary = {
                        'round': event_name,
                        'position': qualifying_position_display,
                        'gapToPole': gap_to_pole,
                        'teammateGap': gaps[driver],
                        'hasTeammateData': not (gaps[driver] == 'N/A')
                    }
                    
                    # Find or create driver entry for this year
                    driver_entry = next(
                        (item for item in timeline_data if item['year'] == year and item['driver'] == driver),
                        None
                    )
                    
                    if driver_entry is None:
                        driver_entry = {
                            'year': year,
                            'driver': driver,
                            'team': team,
                            'events': [],
                            'positions': [],
                            'gapToPole_values': [],
                            'teammateGap_values': [],
                            'completeDataCount': 0,
                            'totalEvents': 0
                        }
                        timeline_data.append(driver_entry)
                    
                    driver_entry['events'].append(event_summary)
                    driver_entry['positions'].append(qualifying_position_value)
                    driver_entry['totalEvents'] += 1
                    
                    if gap_to_pole is not None and gap_to_pole != 'N/A':
                        driver_entry['gapToPole_values'].append(gap_to_pole)
                    if gaps[driver] is not None and gaps[driver] != 'N/A':
                        driver_entry['teammateGap_values'].append(gaps[driver])
                        driver_entry['completeDataCount'] += 1
    
    # Calculate final averages
    for entry in timeline_data:
        # Only include positions that are not NaN
        valid_positions = [pos for pos in entry['positions'] if not pd.isna(pos)]
        entry['avgQualifyingPosition'] = np.mean(valid_positions) if valid_positions else np.nan
    
         #Only include valid gap to pole times (exclude 'N/A' and NaN)
        valid_gaps_to_pole = [gap for gap in entry['gapToPole_values'] if not pd.isna(gap) and gap != 'N/A']
        entry['avgGapToPole'] = np.mean(valid_gaps_to_pole) if valid_gaps_to_pole else np.nan
    
        # Only include teammate gaps where there was actually a comparison
        valid_teammate_gaps = [gap for gap in entry['teammateGap_values'] if not pd.isna(gap) and gap != 'N/A']
        entry['avgTeammateGap'] = np.mean(valid_teammate_gaps) if valid_teammate_gaps else np.nan
    
        entry['dataCompleteness'] = entry['completeDataCount'] / entry['totalEvents'] if entry['totalEvents'] > 0 else 0
        
        # Clean up intermediate calculation fields
        del entry['positions']
        del entry['gapToPole_values']
        del entry['teammateGap_values']
        del entry['completeDataCount']
        del entry['totalEvents']
    
    return timeline_data

In [48]:
career_timeline_data = process_qualifying_data(quali_data)


In [49]:
career_timeline_data

[{'year': 2019,
  'driver': 'A GIOVINAZZI',
  'team': 'Alfa Romeo Racing',
  'events': [{'round': 'Abu Dhabi Grand Prix',
    'position': 17.0,
    'gapToPole': 3.335000000000008,
    'teammateGap': -0.26899999999999125,
    'hasTeammateData': True},
   {'round': 'Australian Grand Prix',
    'position': 14.0,
    'gapToPole': 2.2279999999999944,
    'teammateGap': 0.4000000000000057,
    'hasTeammateData': True},
   {'round': 'Austrian Grand Prix',
    'position': 8.0,
    'gapToPole': 1.176000000000002,
    'teammateGap': 0.01300000000000523,
    'hasTeammateData': True},
   {'round': 'Azerbaijan Grand Prix',
    'position': 8.0,
    'gapToPole': 1.929000000000002,
    'teammateGap': 'N/A',
    'hasTeammateData': False},
   {'round': 'Bahrain Grand Prix',
    'position': 16.0,
    'gapToPole': 2.1599999999999966,
    'teammateGap': 1.0039999999999907,
    'hasTeammateData': True},
   {'round': 'Belgian Grand Prix',
    'position': 15.0,
    'gapToPole': 3.117999999999995,
    'teammat

In [50]:
year_list = []
for dict in career_timeline_data:
    year_list.append(dict['year'])
set(year_list)

{2018, 2019, 2020, 2021, 2022}

In [51]:
len(set(entry['driver'] for entry in career_timeline_data))

33

In [52]:
incomplete_teammate_data = []

for entry in career_timeline_data:
    missing_events = [event['round'] for event in entry['events'] 
                     if not event['hasTeammateData']]
    
    if missing_events:  
        incomplete_teammate_data.append({
            'Year': entry['year'],
            'Driver': entry['driver'],
            'Team': entry['team'],
            'Missing Events': missing_events,
            'Total Missing Events': len(missing_events)
        })
incomplete_teammate_data


[{'Year': 2019,
  'Driver': 'A GIOVINAZZI',
  'Team': 'Alfa Romeo Racing',
  'Missing Events': ['Azerbaijan Grand Prix', 'Chinese Grand Prix'],
  'Total Missing Events': 2},
 {'Year': 2019,
  'Driver': 'K RAIKKONEN',
  'Team': 'Alfa Romeo Racing',
  'Missing Events': ['Azerbaijan Grand Prix', 'Chinese Grand Prix'],
  'Total Missing Events': 2},
 {'Year': 2019,
  'Driver': 'C LECLERC',
  'Team': 'Ferrari',
  'Missing Events': ['German Grand Prix'],
  'Total Missing Events': 1},
 {'Year': 2019,
  'Driver': 'S VETTEL',
  'Team': 'Ferrari',
  'Missing Events': ['German Grand Prix'],
  'Total Missing Events': 1},
 {'Year': 2019,
  'Driver': 'K MAGNUSSEN',
  'Team': 'Haas F1 Team',
  'Missing Events': ['Japanese Grand Prix'],
  'Total Missing Events': 1},
 {'Year': 2019,
  'Driver': 'R GROSJEAN',
  'Team': 'Haas F1 Team',
  'Missing Events': ['Japanese Grand Prix'],
  'Total Missing Events': 1},
 {'Year': 2019,
  'Driver': 'L NORRIS',
  'Team': 'McLaren',
  'Missing Events': ['Brazilian Gran

### Dashboard 


In [54]:
def create_driver_dashboard(timeline_data):
    """
    Create an interactive dashboard showing driver performance across years
    
    Args:
        timeline_data: List of dictionaries containing processed qualifying data
    """
    # Convert timeline data to DataFrame for easier manipulation
    df = pd.DataFrame(timeline_data)
    
    # Create year selector
    year_selector = pn.widgets.Select(
        name='Year',
        options=sorted(df['year'].unique().tolist()),
        width=200
    )
    
    # Create driver selector based on year
    def get_drivers(year):
        return sorted(df[df['year'] == year]['driver'].unique().tolist())
    
    driver_selector = pn.widgets.Select(
        name='Driver',
        options=get_drivers(year_selector.value),
        width=200
    )
    
    # Update driver options when year changes
    def update_drivers(event):
        driver_selector.options = get_drivers(event.new)
    
    year_selector.param.watch(update_drivers, 'value')
    
    # Create performance graph
    def create_performance_plot(year, driver):
        driver_data = df[(df['year'] == year) & (df['driver'] == driver)].iloc[0]
        events_df = pd.DataFrame(driver_data['events'])
        
        # Create position plot
        position_plot = events_df.hvplot.line(
            'round', 'position', 
            title=f'Qualifying Positions - {driver} ({year})',
            width=800, height=300,
            ylim=(20, 1),  # Reverse y-axis to show P1 at top
            label='Position'
        )
        
        return position_plot
    
    # Create metrics panel
    def create_metrics_panel(year, driver):
        driver_data = df[(df['year'] == year) & (df['driver'] == driver)].iloc[0]
        
        metrics = pn.Column(
            pn.pane.Markdown(f"### {driver} - {driver_data['team']} ({year})"),
            pn.pane.Markdown(f"**Average Qualifying Position:** {driver_data['avgQualifyingPosition']:.2f}"),
            pn.pane.Markdown(f"**Average Gap to Pole:** {driver_data['avgGapToPole']:.3f}s"),
            pn.pane.Markdown(f"**Average Gap to Teammate:** {driver_data['avgTeammateGap']:.3f}s"),
            width=400
        )
        return metrics
    
    # Create interactive dashboard
    @pn.depends(year_selector.param.value, driver_selector.param.value)
    def create_dashboard(year, driver):
        if not year or not driver:
            return pn.Column(pn.pane.Markdown("Please select a year and driver"))
        
        plot = create_performance_plot(year, driver)
        metrics = create_metrics_panel(year, driver)
        
        return pn.Column(
            pn.Row(metrics),
            pn.Row(plot),
            pn.Row(pn.pane.Markdown("#### Event Details"),
                  # Add event-specific details here
            )
        )
    
    # Combine all elements
    dashboard = pn.Column(
        pn.Row(year_selector, driver_selector),
        create_dashboard
    )
    
    return dashboard

In [55]:
create_driver_dashboard(career_timeline_data)

UFuncTypeError: ufunc 'greater' did not contain a loop with signature matching types (<class 'numpy.dtypes.Float64DType'>, <class 'numpy.dtypes.StrDType'>) -> None

Column
    [0] Row
        [0] Select(name='Year', options=[2018, 2019, 2020, ...], value=2018, width=200)
        [1] Select(name='Driver', options=['B HARTLEY', ...], value='B HARTLEY', width=200)
    [1] ParamFunction(function, _pane=Column, defer_load=False)